In [ ]:
###############################################################################
import aliases
from everest.utilities import textutils
# tuple(textutils.find_all('', path='../everest/ptolemaic'))
# textutils.replace_all('Schematic', 'Compound', path='../everest')

In [ ]:
from everest.ptolemaic.essence import Essence

class MyClass(metaclass=Essence):

    __mroclasses__ = dict(
        A=(),
        B=('A',),
        )

In [ ]:
MyClass.B

In [ ]:
from everest.algebraic import sett, mapp, chora
from everest.ptolemaic.system import System

In [ ]:
class MyChora(chora.Choret, metaclass=System):

    lower: POS[int]
    upper: POS[int] = None

    @classmethod
    def __parameterise__(cls, /, *args, **kwargs):
        params = super().__parameterise__(*args, **kwargs)
        if params.upper is None:
            params.upper = params.lower
            params.lower = 0
        return params

    def _incise_handle_int_(self, incisor: int, /):
        return chora.IncisionStyle.RETRIEVE(incisor)

    def _incise_handle_slice_(self, incisor: slice, /):
        start, stop, step = incisor.start, incisor.stop, incisor.step
        if step is not None:
            raise ValueError(incisor)
        return chora.IncisionStyle.SLYCE((start, stop))

    def _incise_handle_tuple(self, incisor: tuple, /):
        return chora.IncisionStyle.SLYCE(incisor)

    def _incise_handle_ellipsis_(self, incisor: type(Ellipsis), /):
        return chora.IncisionStyle.TRIVIAL(incisor)

    def _incise_retrieve_int_(self, incisor: int, /):
        return self.lower + incisor if incisor >= 0 else self.upper + incisor

    def _incise_slyce_tuple_(self, lower, upper, /):
        slower, supper = self.params
        return self.__ptolemaic_class__(
            slower if lower is None else \
                (slower if lower >= 0 else supper) + lower,
            supper if upper is None else \
                (slower if upper >= 0 else supper) + upper,
            )

    def __iter__(self, /):
        return iter(range(*self.params))

    def _contains_(self, other, /):
        return self.lower <= other <= self.upper


achora = MyChora(3, 9)
bchora = MyChora(5, 15)
cchora = bchora @ achora
cchora[3:]

In [ ]:
class MyClass(chora.Chorelle, metaclass=System):

    chora: chora.Chora

    def _incise_slyce_(self, incisor, /):
        return self.__ptolemaic_class__(super()._incise_slyce_(incisor))


dchora = MyClass(cchora)
assert dchora[...] is dchora
dchora[3:]

In [ ]:
echora = chora.MultiChora(achora, bchora)
display(echora[0, 2:])
display(echora[0, 2:][0])
(3, 7) in echora